# Kütüphaneler

In [20]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.preprocessing import RobustScaler
pd.set_option('display.max_columns', None)
from unidecode import unidecode
from catboost import CatBoostRegressor

# Dosyalar

In [21]:
train = pd.read_csv("train.csv")
hava = pd.read_csv("hava.csv",sep=";")
test = pd.read_csv("test.csv")

# Atılanlar

In [22]:
test_submission_kolon = test["KESİNTİ_NO"].copy()
test.drop(["BİLDİRİME_GÖRE","KADEME","KESİNTİ_NO","OutageID"],axis=1,inplace=True)
train.drop(["BİLDİRİME_GÖRE","KADEME","KESİNTİ_NO","OutageID","KOD_NO"],axis=1,inplace=True)

# Toplam arama sayısı 0 olan ayların atılması

In [23]:
train.groupby(train["BAŞLAMA_TARİHİ_VE_ZAMANI"].astype(str).str[3:10])['cagri_count'].sum()

BAŞLAMA_TARİHİ_VE_ZAMANI
01.2022     83939.0
02.2022     58037.0
03.2022     35308.0
04.2022         0.0
05.2021     36563.0
05.2022         0.0
06.2021     81359.0
06.2022        68.0
07.2021    105386.0
08.2021     90795.0
09.2021     41087.0
10.2021     68062.0
11.2021     77213.0
12.2021     82052.0
Name: cagri_count, dtype: float64

**06.2022 tarihinde 68 arama olması biraz tuhaf. Bu yüzden bunu da atıyorum**

In [24]:
train = train[~train["BAŞLAMA_TARİHİ_VE_ZAMANI"].astype(str).str[3:10].isin(["05.2022","04.2022","06.2022"])]
train.groupby(train["BAŞLAMA_TARİHİ_VE_ZAMANI"].astype(str).str[3:10])['cagri_count'].sum()

BAŞLAMA_TARİHİ_VE_ZAMANI
01.2022     83939.0
02.2022     58037.0
03.2022     35308.0
05.2021     36563.0
06.2021     81359.0
07.2021    105386.0
08.2021     90795.0
09.2021     41087.0
10.2021     68062.0
11.2021     77213.0
12.2021     82052.0
Name: cagri_count, dtype: float64

# Kontrol

Sıfırdan küçükler var mı

In [25]:
for i in test.select_dtypes(include=["int","float"]).columns:
    print(i,train[i].min(),test[i].min())

KESİNTİ_SÜRESİ 0.000277777 0.000555555
KENTSEL_OG 0 0
KENTSEL_AG 0 0
KENTALTI_OG 0 0
KENTALTI_AG 0 0
KIRSAL_OG 0 0
KIRSAL_AG 0 0
TOPLAM_KENTSEL_OG 0.0 0.0
TOPLAM_KENTSEL_AG 0.0 0.0
TOPLAM_KENTALTI_OG 0.0 0.0
TOPLAM_KENTALTI_AG 0.0 0.0
TOPLAM_KIRSAL_OG 0.0 0.0
TOPLAM_KIRSAL_AG 0.0 0.0


In [26]:
for i in test.columns:
    print(i,train[i].isna().sum(),test[i].isna().sum())

İL 0 0
İLÇE 0 0
ŞEBEKE_UNSURU 0 0
ŞEBEKE_UNSURU_KODU 0 0
KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA 0 0
KAYNAĞA_GÖRE 0 0
SÜREYE_GÖRE 0 0
SEBEBE_GÖRE 0 0
BAŞLAMA_TARİHİ_VE_ZAMANI 0 0
SONA_ERME_TARİHİ_VE_ZAMANI 0 0
KESİNTİ_SÜRESİ 0 0
KENTSEL_OG 0 0
KENTSEL_AG 0 0
KENTALTI_OG 0 0
KENTALTI_AG 0 0
KIRSAL_OG 0 0
KIRSAL_AG 0 0
TOPLAM_KENTSEL_OG 0 0
TOPLAM_KENTSEL_AG 0 0
TOPLAM_KENTALTI_OG 0 0
TOPLAM_KENTALTI_AG 0 0
TOPLAM_KIRSAL_OG 0 0
TOPLAM_KIRSAL_AG 0 0


# Gün Dönemi

In [27]:
train.insert(11, 'Kesinti_Gun_Donemi', train["BAŞLAMA_TARİHİ_VE_ZAMANI"].astype(str).str[11:13])
train.loc[train["Kesinti_Gun_Donemi"].isin(["00","01","02","03","04","05"]),"Kesinti_Gun_Donemi"] = "Gece"
train.loc[train["Kesinti_Gun_Donemi"].isin(["06","07","08","09","10","11"]),"Kesinti_Gun_Donemi"] = "Sabah"
train.loc[train["Kesinti_Gun_Donemi"].isin(["12","13"]),"Kesinti_Gun_Donemi"] = "Öğlen"
train.loc[train["Kesinti_Gun_Donemi"].isin(["14","15","16"]),"Kesinti_Gun_Donemi"] = "İkindi"
train.loc[train["Kesinti_Gun_Donemi"].isin(["17","18","19"]),"Kesinti_Gun_Donemi"] = "Akşamüstü"
train.loc[train["Kesinti_Gun_Donemi"].isin(["20","21","22","23"]),"Kesinti_Gun_Donemi"] = "Akşam"
#---------------------------------------------------------------------------------------
test.insert(11, 'Kesinti_Gun_Donemi', test["BAŞLAMA_TARİHİ_VE_ZAMANI"].astype(str).str[11:13])
test.loc[test["Kesinti_Gun_Donemi"].isin(["00","01","02","03","04","05"]),"Kesinti_Gun_Donemi"] = "Gece"
test.loc[test["Kesinti_Gun_Donemi"].isin(["06","07","08","09","10","11"]),"Kesinti_Gun_Donemi"] = "Sabah"
test.loc[test["Kesinti_Gun_Donemi"].isin(["12","13"]),"Kesinti_Gun_Donemi"] = "Öğlen"
test.loc[test["Kesinti_Gun_Donemi"].isin(["14","15","16"]),"Kesinti_Gun_Donemi"] = "İkindi"
test.loc[test["Kesinti_Gun_Donemi"].isin(["17","18","19"]),"Kesinti_Gun_Donemi"] = "Akşamüstü"
test.loc[test["Kesinti_Gun_Donemi"].isin(["20","21","22","23"]),"Kesinti_Gun_Donemi"] = "Akşam"

# Tarih Kolonu

In [28]:
train["BAŞLAMA_TARİHİ_VE_ZAMANI"] = train["BAŞLAMA_TARİHİ_VE_ZAMANI"].str[:10].apply(lambda x: dt.datetime.strptime(x,'%d.%m.%Y'))
test["BAŞLAMA_TARİHİ_VE_ZAMANI"] = test["BAŞLAMA_TARİHİ_VE_ZAMANI"].str[:10].apply(lambda x: dt.datetime.strptime(x,'%d.%m.%Y'))
hava["BAŞLAMA_TARİHİ_VE_ZAMANI"] = hava["BAŞLAMA_TARİHİ_VE_ZAMANI"].str[:10].apply(lambda x: dt.datetime.strptime(x,'%d.%m.%Y'))

train["SONA_ERME_TARİHİ_VE_ZAMANI"] = train["SONA_ERME_TARİHİ_VE_ZAMANI"].str[:10].apply(lambda x: dt.datetime.strptime(x,'%d.%m.%Y'))
test["SONA_ERME_TARİHİ_VE_ZAMANI"] = test["SONA_ERME_TARİHİ_VE_ZAMANI"].str[:10].apply(lambda x: dt.datetime.strptime(x,'%d.%m.%Y'))

# Nüfus

Uzun yoldan yapmıştım kolaylık için fikir -> https://www.kaggle.com/code/karakasatarik/get-population-data

In [29]:
izmir_nufus = pd.read_html('https://www.nufusu.com/ilceleri/izmir-ilceleri-nufusu')[0][["İlçe","Toplam Nüfus"]]
manisa_nufus = pd.read_html('https://www.nufusu.com/ilceleri/manisa-ilceleri-nufusu')[0][["İlçe","Toplam Nüfus"]]
nufus = pd.concat([izmir_nufus,manisa_nufus])
nufus["İlçe"] = nufus["İlçe"].apply(unidecode).str.lower()
train["İLÇE"] = train["İLÇE"].apply(unidecode).str.lower()
test["İLÇE"] = test["İLÇE"].apply(unidecode).str.lower()
train = pd.merge(train,nufus,how="left",left_on="İLÇE",right_on="İlçe")
test = pd.merge(test,nufus,how="left",left_on="İLÇE",right_on="İlçe")
train["Toplam Nüfus"] = train["Toplam Nüfus"] * 1000
train["Toplam Nüfus"] = train["Toplam Nüfus"].astype("int")
test["Toplam Nüfus"] = test["Toplam Nüfus"] * 1000
test["Toplam Nüfus"] = test["Toplam Nüfus"].astype("int")

# Hava Durumu

In [30]:
train = pd.merge(train,hava,how="left",on="BAŞLAMA_TARİHİ_VE_ZAMANI")
test = pd.merge(test,hava,how="left",on="BAŞLAMA_TARİHİ_VE_ZAMANI")

# Tarih Parçala

In [31]:
train["BAŞLAMA_TARİHİ_VE_ZAMANI_AY"] = train["BAŞLAMA_TARİHİ_VE_ZAMANI"].dt.month.astype("str")
train["BAŞLAMA_TARİHİ_VE_ZAMANI_GUN"] = train["BAŞLAMA_TARİHİ_VE_ZAMANI"].dt.day.astype("str")
train["BAŞLAMA_TARİHİ_VE_ZAMANI_YIL"] = train["BAŞLAMA_TARİHİ_VE_ZAMANI"].dt.year.astype("str")
train['başlama_haftasi'] = train["BAŞLAMA_TARİHİ_VE_ZAMANI"].dt.dayofweek.astype("str")
train['bitme_haftasi'] = train["SONA_ERME_TARİHİ_VE_ZAMANI"].dt.dayofweek.astype("str")
##
test["BAŞLAMA_TARİHİ_VE_ZAMANI_AY"] = test["BAŞLAMA_TARİHİ_VE_ZAMANI"].dt.month.astype("str")
test["BAŞLAMA_TARİHİ_VE_ZAMANI_GUN"] = test["BAŞLAMA_TARİHİ_VE_ZAMANI"].dt.day.astype("str")
test["BAŞLAMA_TARİHİ_VE_ZAMANI_YIL"] = test["BAŞLAMA_TARİHİ_VE_ZAMANI"].dt.year.astype("str")
test['başlama_haftasi'] = test["BAŞLAMA_TARİHİ_VE_ZAMANI"].dt.dayofweek.astype("str")
test['bitme_haftasi'] = test["SONA_ERME_TARİHİ_VE_ZAMANI"].dt.dayofweek.astype("str")
##
train.drop(["BAŞLAMA_TARİHİ_VE_ZAMANI"],axis=1,inplace=True)
test.drop(["BAŞLAMA_TARİHİ_VE_ZAMANI"],axis=1,inplace=True)
train.drop(["SONA_ERME_TARİHİ_VE_ZAMANI"],axis=1,inplace=True)
test.drop(["SONA_ERME_TARİHİ_VE_ZAMANI"],axis=1,inplace=True)

# Mevsim

In [32]:
train.loc[train["BAŞLAMA_TARİHİ_VE_ZAMANI_AY"].isin(["12","1","2"]),"mevsim"] = "Kış"
train.loc[train["BAŞLAMA_TARİHİ_VE_ZAMANI_AY"].isin(["3","4","5"]),"mevsim"] = "İlkbahar"
train.loc[train["BAŞLAMA_TARİHİ_VE_ZAMANI_AY"].isin(["6","7","8"]),"mevsim"] = "Yaz"
train.loc[train["BAŞLAMA_TARİHİ_VE_ZAMANI_AY"].isin(["9","10","11"]),"mevsim"] = "Sonbahar"

test.loc[test["BAŞLAMA_TARİHİ_VE_ZAMANI_AY"].isin(["12","1","2"]),"mevsim"] = "Kış"
test.loc[test["BAŞLAMA_TARİHİ_VE_ZAMANI_AY"].isin(["3","4","5"]),"mevsim"] = "İlkbahar"
test.loc[test["BAŞLAMA_TARİHİ_VE_ZAMANI_AY"].isin(["6","7","8"]),"mevsim"] = "Yaz"
test.loc[test["BAŞLAMA_TARİHİ_VE_ZAMANI_AY"].isin(["9","10","11"]),"mevsim"] = "Sonbahar"

# Encoding

In [33]:
for i in train.select_dtypes(include=["object"]).columns:
    unique_degerler = set(train[i].unique()).union(test[i].unique()) 
    sozluk = {k: v for v, k in enumerate(unique_degerler)}
    train[i] = train[i].map(sozluk)
    test[i] = test[i].map(sozluk)

# Model Değişkenleri

In [34]:
train_x = train.drop(["cagri_count"],axis=1)
train_y = train["cagri_count"]

# RobustScaler

In [35]:
scaler = RobustScaler().fit(train_x)
train_x = scaler.transform(train_x)
test = scaler.transform(test)

# Model

In [36]:
model = CatBoostRegressor(n_estimators=3307,max_depth= 3, max_bin= 66,learning_rate=0.059, random_state = 6, verbose=False) #3307
model.fit(train_x, train_y)
print(model.score(train_x, train_y))
tahmin = model.predict(test)
tahmin[tahmin < 0] = 0.0
tahmin = tahmin.round(0).astype("float")

0.7553370809039712


In [38]:
final = pd.concat([test_submission_kolon,pd.DataFrame(tahmin)],axis=1)
final.columns = ["KESİNTİ_NO","cagri_count"]
final.to_csv("submission.csv",index=False)